In [14]:
from skimage.feature import hog
from skimage import data, color, exposure
from skimage.io import imread

In [15]:
import pandas as pd

In [16]:
df = pd.read_csv("./local_path.csv")

In [17]:
PREFIX = "/data/damoncrockett/"

In [18]:
df.local_path = df.local_path.map(lambda x: PREFIX + x[33:])

In [ ]:
import numpy as np

In [ ]:
vector_list = []

counter = -1
for i in range(len(df)):
    img = imread(df.local_path.loc[i])
    img_gray = color.rgb2gray(img)
    fd = hog(img_gray, orientations=9, pixels_per_cell=(8, 8),cells_per_block=(4, 4))
    vector_list.append(fd)
    counter+=1
    print counter
    
X = np.vstack(vector_list)

In [ ]:
from sklearn.manifold import TSNE as tsne

tsne = tsne(n_components=2)
tsne.fit(X)
subspace_tsne = pd.DataFrame(tsne.fit_transform(X),columns=["x","y"])

In [ ]:
num_bins = 96

In [ ]:
subspace_tsne['grid_x'] = pd.cut(subspace_tsne['x'],num_bins,labels=False)
subspace_tsne['grid_y'] = pd.cut(subspace_tsne['y'],num_bins,labels=False)

In [ ]:
subspace_tsne['local_path'] = df.local_path

In [ ]:
thumb_side = 128

In [ ]:
from PIL import Image

n = len(subspace_tsne)

grid_side = num_bins

px_w = thumb_side * grid_side
px_h = thumb_side * grid_side

canvas = Image.new('RGB',(px_w,px_h),(50,50,50))

for i in range(n):
    im = Image.open(subspace_tsne.local_path.loc[i])
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    x = subspace_tsne.grid_x.loc[i] * thumb_side
    y = subspace_tsne.grid_y.loc[i] * thumb_side
    canvas.paste(im,(x,y))
    
canvas.save("./hog.png")